In [ ]:
# Import required modules
import pandas as pd
import re                                           # regular expressions
import nltk
import nltk.corpus as corpus                        # nltk package for reading files
nltk.download('stopwords')
from nltk.corpus import stopwords as stopwords      # words such as 'the','be','an', that don't significantly impact the sentiment
from nltk.stem import WordNetLemmatizer             # Groups words
from nltk.tokenize import word_tokenize             # Splits text into tokens
import collections
# Will be used later on
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
# import sklearn
# import numpy as np

In [ ]:
# Filter out all unlabelled sentiment
p_df = pd.read_csv('/home/mia/Documents/College/CS4811/TwitterNLP/tweets_labelled.csv', sep=';',names=['created_at','text','sentiment']).query('sentiment == "positive"')
n_df = pd.read_csv('/home/mia/Documents/College/CS4811/TwitterNLP/tweets_labelled.csv', sep=';',names=['created_at','text','sentiment']).query('sentiment == "negative"')

# Debug Print
# display(p_df)
# display(n_df)

# Combine dataframes
frames = [p_df, n_df]
df = pd.concat(frames)

# Sort by date
# https://www.geeksforgeeks.org/how-to-sort-a-pandas-dataframe-by-date/
df['created_at'] = pd.to_datetime(df['created_at'])
df = df.sort_values(by="created_at",ascending=True)
# Debug Print
display(df)


In [ ]:
# Grab the top 25 stocks
ticker_pattern = re.compile(r'(^\$[A-Z]+|^\$ES_F)')

# Break the string into a dictionary
ticker_freq_dict = collections.defaultdict(int)

for text in df['text']:
    for word in text.split():
        if ticker_pattern.fullmatch(word) is not None:
            ticker_freq_dict[word[1:]] += 1

ticker_freq_df = pd.DataFrame.from_dict(
    ticker_freq_dict, orient='index', columns=['freq'])\
    .sort_values('freq', ascending=False).head(25)
display(ticker_freq_df)    

In [35]:
# Create a dictionary of strings to save each within the dictionary
ticker_dict = collections.defaultdict(str)

for text in df['text']:
    for ticker in ticker_freq_df.index:
        if text.find(ticker) > 0:
            if ticker not in ticker_dict:
                ticker_dict[ticker] = [text]
                break
        # else:
        #     ticker_dict[ticker].append(text)


In [36]:
# Print out the dictionary to json file for debugging purposes
import json

with open('ticker_dict.json','w') as fout:
    json.dump(ticker_dict, fout, indent=4)

# Confirm there are a correct number of elements in each value set
print(len(ticker_dict['APPL']))

0


In [ ]:
# Return a weight from 0-1 based on the percentage of positive tweets
